In [2]:
import pandas as pd
import numpy 
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.metrics import roc_auc_score
import pickle

In [12]:
# load sample data
home_stat = pd.read_csv("/Users/paraspokharel/Programming/pitchProphet/pitchProphet/data/pre_processing/processed/home_stats_43rows.csv")
away_stat = pd.read_csv("/Users/paraspokharel/Programming/pitchProphet/pitchProphet/data/pre_processing/processed/away_stats_43rows.csv")
match_stat = pd.read_csv("/Users/paraspokharel/Programming/pitchProphet/pitchProphet/data/pre_processing/processed/match_info_with_labels_43rows.csv")


In [13]:
# prepare training data


if "Unnamed: 0" in home_stat.columns: 
    home_stat.drop(columns = ["Unnamed: 0"], inplace = True)
if "Unnamed: 0" in away_stat.columns: 
    away_stat.drop(columns = ["Unnamed: 0"], inplace = True)

# rename columns
home_stat.columns = ["h" + col for col in home_stat.columns]
away_stat.columns = ["a" + col for col in away_stat.columns]
x_df = pd.concat([home_stat, away_stat], axis = 1)
y_df = match_stat[["label"]]

x_df = x_df.apply(pd.to_numeric)
y_df = y_df.apply(pd.to_numeric)

x_train, x_test = train_test_split(x_df, test_size = 0.3, random_state = 42)
y_train, y_test = train_test_split(y_df, test_size = 0.3, random_state = 42)

print(x_train,y_train)

    hGls_mean  hGls_std    hGls_trend  hAst_mean  hAst_std    hAst_trend  \
19   0.666667  0.577350 -5.000000e-01   0.333333  0.577350 -5.000000e-01   
17   1.750000  0.957427 -1.000000e-01   1.500000  0.577350 -2.967196e-16   
13   1.000000  0.816497  4.000000e-01   1.000000  0.816497  4.000000e-01   
40   1.000000  0.816497  2.000000e-01   0.250000  0.500000  1.000000e-01   
15   3.250000  1.258306  3.000000e-01   2.500000  0.577350  4.000000e-01   
9    1.333333  1.527525  5.000000e-01   1.000000  1.000000  5.000000e-01   
16   1.750000  1.500000  3.000000e-01   1.500000  1.290994 -2.967196e-16   
30   0.666667  0.577350  5.000000e-01   0.000000  0.000000 -0.000000e+00   
26   0.666667  0.577350  5.000000e-01   0.333333  0.577350  5.000000e-01   
0    1.500000  0.707107  1.000000e+00   1.500000  0.707107  1.000000e+00   
29   0.333333  0.577350 -5.000000e-01   0.000000  0.000000 -0.000000e+00   
41   2.250000  0.957427  3.000000e-01   1.750000  0.500000  3.000000e-01   
5    1.00000

In [14]:
# train boost

model = xgb.XGBClassifier(objective='multi:softmax', num_class=3, learning_rate=0.1, max_depth=6, n_estimators=100)

# Fit the model with early stopping and evaluation metric
model.fit(x_train, y_train,
          eval_set=[(x_train, y_train), (x_test, y_test)],    
          verbose=False) 
# TODO: optimize

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, num_class=3,
              num_parallel_tree=None, ...)

In [15]:
# evaluate
eval_result = model.evals_result()
pred_y = model.predict_proba(x_test)
auc =  roc_auc_score(y_test, pred_y, multi_class = "ovo")

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed

In [ ]:
# download model (pickle converts to byte stream)
pickle.dump(model, open(r"../xgb_model.pkl", "wb"))